# Business Understanding

In [1]:
import pandas as pd
import numpy as np

print('Pandas:', pd.__version__)
print('Numpy:',np.__version__)
df = pd.read_csv('NBA_datasets/games.csv') # read in the csv file
df.head(n = 100)

Pandas: 1.4.2
Numpy: 1.21.5


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005,Final,1610612748,1610612750,2021,1610612748,104.0,0.398,0.760,...,23.0,53.0,1610612750,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006,Final,1610612741,1610612739,2021,1610612741,101.0,0.443,0.933,...,20.0,46.0,1610612739,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007,Final,1610612759,1610612754,2021,1610612759,108.0,0.412,0.813,...,28.0,52.0,1610612754,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008,Final,1610612744,1610612749,2021,1610612744,122.0,0.484,0.933,...,33.0,55.0,1610612749,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009,Final,1610612743,1610612761,2021,1610612743,115.0,0.551,0.750,...,32.0,39.0,1610612761,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2009-10-04,10900004,Final,1610612752,1610612751,2009,1610612752,115.0,0.432,0.743,...,23.0,45.0,1610612751,107.0,0.441,0.732,0.375,15.0,47.0,1
9996,2009-10-04,10900006,Final,1610612750,1610612749,2009,1610612750,117.0,0.508,0.893,...,19.0,49.0,1610612749,103.0,0.443,0.684,0.350,25.0,24.0,1
9997,2009-10-04,10900005,Final,1610612744,1610612746,2009,1610612744,108.0,0.437,0.743,...,24.0,43.0,1610612746,101.0,0.493,0.739,0.435,27.0,43.0,1
9998,2009-10-02,10900002,Final,1610612754,1610612741,2009,1610612754,95.0,0.431,0.839,...,22.0,38.0,1610612741,104.0,0.417,0.861,0.250,17.0,43.0,0
